# 变量简介

TensorFlow **变量**是用于表示程序处理的共享持久状态的推荐方法。本指南介绍在 TensorFlow 中如何创建、更新和管理 `tf.Variable` 的实例。

变量通过 `tf.Variable` 类进行创建和跟踪。`tf.Variable` 表示张量，对它执行运算可以改变其值。利用特定运算可以读取和修改此张量的值。更高级的库（如 [`tf.keras`](https://www.tensorflow.org/api_docs/python/tf/keras)）使用 `tf.Variable` 来存储模型参数。 

## 设置

本笔记本讨论变量布局。如果要查看变量位于哪一个设备上，请取消注释这一行代码。

In [1]:
import tensorflow as tf

# 此设置可以查看变量的放置位置（cpu/gpu），不需要的话可以注释
tf.debugging.set_log_device_placement(True)

## 创建变量

要创建变量，请提供一个初始值。`tf.Variable` 与初始值的 `dtype` 相同。

In [2]:
my_tensor = tf.constant([[1.0, 2.0], [3.0, 4.0]])
my_variable = tf.Variable(my_tensor)

# 变量可以是各种类型，就像张量一样
bool_variable = tf.Variable([False, False, False, True])
# 复数
complex_variable = tf.Variable([5 + 4j, 6 + 1j])

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0


变量与张量的定义方式和操作行为都十分相似，实际上，它们都是 `tf.Tensor` 支持的一种数据结构。与张量类似，变量也有 `dtype` 和形状，并且可以导出至 NumPy。

In [3]:
print("形状: ",my_variable.shape)
print("DType: ",my_variable.dtype)
print("转为 NumPy: ", my_variable.numpy)

形状:  (2, 2)
DType:  <dtype: 'float32'>
转为 NumPy:  Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
<bound method BaseResourceVariable.numpy of <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>>


大部分张量运算在变量上也可以按预期运行，不过变量无法重构形状。

In [4]:
print("一个变量:",my_variable)
print("\n被视为张量:", tf.convert_to_tensor(my_variable))
print("\n最高价值指数(Index of highest value):", tf.argmax(my_variable))

# 这会创建一个新的张量；它不会重构变量。
print("\n复制并创建一个新张量: ", tf.reshape(my_variable, ([1,4])))

一个变量: Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0

被视为张量: tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ArgMax in device /job:localhost/replica:0/task:0/device:GPU:0

最高价值指数(Index of highest value): tf.Tensor([1 1], shape=(2,), dtype=int64)
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0

复制并创建一个新张量:  tf.Te

如上所述，变量由张量提供支持。您可以使用 [`tf.Variable.assign`](https://www.tensorflow.org/api_docs/python/tf/Variable#assign) 重新分配张量。调用 `assign`（通常）不会分配新张量，而会重用现有张量的内存。

In [5]:
a = tf.Variable([2.0, 3.0])
# 这将保持相同的 dtype，float32
a.assign([1, 2]) 
# 不允许，因为它会调整变量的大小：
try:
  a.assign([1.0, 2.0, 3.0])
except Exception as e:
  print(f"{type(e).__name__}: {e}")

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
ValueError: Cannot assign value to variable ' Variable:0': Shape mismatch.The variable shape (2,), and the assigned value shape (3,) are incompatible.


如果在运算中像使用张量一样使用变量，那么通常会对支持张量执行运算。

从现有变量创建新变量会复制支持张量。两个变量不能共享同一内存空间。

In [7]:
a = tf.Variable([2.0, 3.0])
# 根据 a 的值创建 b
b = tf.Variable(a)
a.assign([5, 6])

# a 和 b 不同
print(a.numpy())
print(b.numpy())

# 还有其他版本的 assign
print(a.assign_add([2,3]).numpy())  # [7. 9.]
print(a.assign_sub([7,9]).numpy())  # [0. 0.]

[5. 6.]
[2. 3.]
[7. 9.]
[0. 0.]


## 生命周期、命名和监视

在基于 Python 的 TensorFlow 中，`tf.Variable` 实例与其他 Python 对象的生命周期相同。如果没有对变量的引用，则会自动将其解除分配。

为了便于跟踪和调试，您还可以为变量命名。两个变量可以使用相同的名称。

In [6]:
# 创建 a 和 b；它们具有相同的值，但由不同的张量支持(but are backed by different tensors)。
a = tf.Variable(my_tensor, name="Mark")
# 一个同名但值不同的新变量
# 注意标量加法是广播的
b = tf.Variable(my_tensor + 1, name="Mark")

# 尽管名称相同，但它们在元素上是不相等的
print("my_tensor:",my_tensor)
print("\n")
print(a == b)

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
my_tensor: tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)


Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Equal in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[False False]
 [False False]], shape=(2, 2), dtype=bool)


保存和加载模型时会保留变量名。默认情况下，模型中的变量会自动获得唯一变量名，所以除非您希望自行命名，否则不必多此一举。

虽然变量对微分很重要，但某些变量不需要进行微分。在创建时，通过将 `trainable` 设置为 False 可以关闭梯度。例如，训练计步器就是一个不需要梯度的变量。

In [7]:
step_counter = tf.Variable(1, trainable=False)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0


## 放置变量和张量

为了提高性能，TensorFlow 会尝试将张量和变量放在与其 `dtype` 兼容的最快设备上。这意味着如果有 GPU，那么大部分变量都会放置在 GPU 上。

不过，我们可以重写变量的位置。在以下代码段中，即使存在可用的 GPU，我们也可以将一个浮点张量和一个变量放置在 CPU 上。通过打开设备分配日志记录（参阅[设置](#scrollTo=xZoJJ4vdvTrD)），可以查看变量的所在位置。

注：虽然可以手动放置变量，但使用[分布策略](distributed_training)是一种可优化计算的更便捷且可扩展的方式。

如果在有 GPU 和没有 GPU 的不同后端上运行此笔记本，则会看到不同的记录。*请注意，必须在会话开始时打开设备布局记录。*

In [8]:
with tf.device('CPU:0'):

  # 创建一些张量
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
  c = tf.matmul(a, b)

print(c)

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


- [`tf.device('CPU:0')`](https://www.tensorflow.org/api_docs/python/tf/device): 指定在此上下文中创建/执行的操作的设备

您可以将变量或张量的位置设置在一个设备上，然后在另一个设备上执行计算。但这样会产生延迟，因为需要在两个设备之间复制数据。

不过，如果您有多个 GPU 工作进程，但希望变量只有一个副本，则可以这样做。

In [9]:
with tf.device('CPU:0'):
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.Variable([[1.0, 2.0, 3.0]])

with tf.device('GPU:0'):
  # 逐元素乘法(Element-wise multiply)
  k = a * b

print(k)

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[ 1.  4.  9.]
 [ 4. 10. 18.]], shape=(2, 3), dtype=float32)


注：由于 `tf.config.set_soft_device_placement` 默认处于打开状态，所以，即使在没有 GPU 的设备上运行此代码，它也会运行，只不过乘法步骤在 CPU 上执行。

有关分布式训练的详细信息，请参阅[我们的指南](distributed_training)。

## 后续步骤

要了解变量的一般使用方法，请参阅关于[自动分布](autodiff)的指南。